In [1]:
import tensorflow as tf

import numpy as np
import os
import time

print(tf.__version__)

2.2.0


In [4]:
labels = set()

def file2Examples(file_name):
    '''
    Read data files and return input/output pairs
    '''

    examples=[]

    with open(file_name,"r") as f:

        next(f)
        next(f)

        example = [[],[]]

        for line in f:

            input_output_split= line.split()

            if len(input_output_split)==4:
                example[0].append(input_output_split[0])
                example[1].append(input_output_split[-1])
                labels.add(input_output_split[-1])

            elif len(input_output_split)==0:
                examples.append(example)
                example=[[],[]]
            else:
                example=[[],[]]

        f.close()

    return examples

# Extract examples from train, validation, and test files which can be found at 
# https://github.com/davidsbatista/NER-datasets/tree/master/CONLL2003
train_examples = file2Examples("/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/2.Scripts para NER nos documentos/NER-datasets-master/CONLL2003/train.txt")
test_examples = file2Examples("/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/2.Scripts para NER nos documentos/NER-datasets-master/CONLL2003/test.txt")
valid_examples = file2Examples("/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/2.Scripts para NER nos documentos/NER-datasets-master/CONLL2003/valid.txt")

In [5]:
train_examples

[[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
  ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']],
 [['Peter', 'Blackburn'], ['B-PER', 'I-PER']],
 [['BRUSSELS', '1996-08-22'], ['B-LOC', 'O']],
 [['The',
   'European',
   'Commission',
   'said',
   'on',
   'Thursday',
   'it',
   'disagreed',
   'with',
   'German',
   'advice',
   'to',
   'consumers',
   'to',
   'shun',
   'British',
   'lamb',
   'until',
   'scientists',
   'determine',
   'whether',
   'mad',
   'cow',
   'disease',
   'can',
   'be',
   'transmitted',
   'to',
   'sheep',
   '.'],
  ['O',
   'B-ORG',
   'I-ORG',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-MISC',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-MISC',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']],
 [['Germany',
   "'s",
   'representative',
   'to',
   'the',
   'European',
   'Union',
   "'s",
   'veterinary',
   'committee',
 

In [6]:
# create character vocab
all_text = " ".join([" ".join(x[0]) for x in train_examples+valid_examples+test_examples])
vocab = sorted(set(all_text))

# create character/id and label/id mapping
char2idx = {u:i+1 for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
label2idx = {u:i+1 for i, u in enumerate(labels)}
idx2label = np.array(labels)

print(idx2label)
print(char2idx)

{'I-LOC', 'O', 'I-MISC', 'B-LOC', 'B-MISC', 'I-ORG', 'B-ORG', 'I-PER', 'B-PER'}
{' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '=': 29, '?': 30, '@': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, ']': 59, '`': 60, 'a': 61, 'b': 62, 'c': 63, 'd': 64, 'e': 65, 'f': 66, 'g': 67, 'h': 68, 'i': 69, 'j': 70, 'k': 71, 'l': 72, 'm': 73, 'n': 74, 'o': 75, 'p': 76, 'q': 77, 'r': 78, 's': 79, 't': 80, 'u': 81, 'v': 82, 'w': 83, 'x': 84, 'y': 85, 'z': 86}


In [12]:
def split_char_labels(eg):
    '''
    For a given input/output example, break tokens into characters while keeping 
    the same label.
    '''

    tokens = eg[0]
    labels=eg[1]

    input_chars = []
    output_char_labels = []

    for token,label in zip(tokens,labels):

        input_chars.extend([char for char in token])
        input_chars.extend(' ')
        output_char_labels.extend([label]*len(token))
        output_char_labels.extend('O')

    return [[char2idx[x] for x in input_chars[:-1]],np.array([label2idx[x] for x in output_char_labels[:-1]])]
   
train_formatted = [split_char_labels(eg) for eg in train_examples]
test_formatted = [split_char_labels(eg) for eg in test_examples]
valid_formatted = [split_char_labels(eg) for eg in valid_examples]

print(len(train_formatted))
print(len(test_formatted))
print(len(valid_formatted)), len(train_formatted[0])

14985
3682
3464


(None, 2)

In [13]:
# training generator
def gen_train_series():

    for eg in train_formatted:
        yield eg[0],eg[1]

# validation generator
def gen_valid_series():

    for eg in valid_formatted:
        yield eg[0],eg[1]

# test generator
def gen_test_series():

    for eg in test_formatted:
        yield eg[0],eg[1]

# create Dataset objects for train, test and validation sets  
series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_valid = tf.data.Dataset.from_generator(gen_valid_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_test = tf.data.Dataset.from_generator(gen_test_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

BATCH_SIZE = 128
BUFFER_SIZE=1000

# create padded batch series objects for train, test and validation sets
ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_valid = series_valid.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_test = series_test.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)

# print example batches
for input_example_batch, target_example_batch in ds_series_batch_valid.take(1):
    print(input_example_batch)
    print(target_example_batch)

tf.Tensor(
[[34 49 40 ...  0  0  0]
 [43 46 45 ...  0  0  0]
 [54 65 79 ...  0  0  0]
 ...
 [ 3  1 36 ...  0  0  0]
 [40 66  1 ...  0  0  0]
 [35 81 78 ...  0  0  0]], shape=(128, 228), dtype=int32)
tf.Tensor(
[[2 2 2 ... 0 0 0]
 [4 4 4 ... 0 0 0]
 [5 5 5 ... 0 0 0]
 ...
 [2 2 2 ... 0 0 0]
 [2 2 2 ... 0 0 0]
 [9 9 9 ... 0 0 0]], shape=(128, 228), dtype=int32)


In [42]:
vocab_size = len(vocab)+1

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

label_size = len(labels)  

# build LSTM model
def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None],
                                  mask_zero=True),
        tf.keras.layers.LSTM(rnn_units,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(1024),
        tf.keras.layers.Dense(label_size)
        ])
    return model

model = build_model(
    vocab_size = len(vocab)+1,
    label_size=len(labels)+1,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (128, None, 256)          22272     
_________________________________________________________________
lstm_6 (LSTM)                (128, None, 1024)         5246976   
_________________________________________________________________
dense_6 (Dense)              (128, None, 1024)         1049600   
_________________________________________________________________
dense_7 (Dense)              (128, None, 10)           10250     
Total params: 6,329,098
Trainable params: 6,329,098
Non-trainable params: 0
_________________________________________________________________


In [43]:
import os

# define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [44]:
EPOCHS=20  
history = model.fit(ds_series_batch, epochs=EPOCHS, validation_data=ds_series_batch_valid,callbacks=[checkpoint_callback])

Epoch 1/20
117/117 [==============================] - 63s 538ms/step - loss: 0.2174 - sparse_categorical_accuracy: 0.2227 - val_loss: 0.1739 - val_sparse_categorical_accuracy: 0.2313
Epoch 2/20
117/117 [==============================] - 62s 533ms/step - loss: 0.1180 - sparse_categorical_accuracy: 0.2296 - val_loss: 0.1067 - val_sparse_categorical_accuracy: 0.2461
Epoch 3/20
117/117 [==============================] - 64s 543ms/step - loss: 0.0960 - sparse_categorical_accuracy: 0.2377 - val_loss: 0.0978 - val_sparse_categorical_accuracy: 0.2492
Epoch 4/20
117/117 [==============================] - 63s 540ms/step - loss: 0.0887 - sparse_categorical_accuracy: 0.2417 - val_loss: 0.0897 - val_sparse_categorical_accuracy: 0.2521
Epoch 5/20
117/117 [==============================] - 64s 547ms/step - loss: 0.0839 - sparse_categorical_accuracy: 0.2424 - val_loss: 0.0868 - val_sparse_categorical_accuracy: 0.2536
Epoch 6/20
117/117 [==============================] - 62s 534ms/step - loss: 0.0781 -

In [38]:
ds_series_batch
for input_example_batch, target_example_batch in ds_series_batch_valid:
    print(input_example_batch, target_example_batch)

tf.Tensor(
[[34 49 40 ...  0  0  0]
 [43 46 45 ...  0  0  0]
 [54 65 79 ...  0  0  0]
 ...
 [ 3  1 36 ...  0  0  0]
 [40 66  1 ...  0  0  0]
 [35 81 78 ...  0  0  0]], shape=(128, 228), dtype=int32) tf.Tensor(
[[2 2 2 ... 0 0 0]
 [4 4 4 ... 0 0 0]
 [5 5 5 ... 0 0 0]
 ...
 [2 2 2 ... 0 0 0]
 [2 2 2 ... 0 0 0]
 [9 9 9 ... 0 0 0]], shape=(128, 228), dtype=int32)
tf.Tensor(
[[34 61 73 ...  0  0  0]
 [14 35 46 ...  0  0  0]
 [50 48 52 ...  0  0  0]
 ...
 [45 36 54 ...  0  0  0]
 [47 39 40 ...  0  0  0]
 [34 36 45 ...  0  0  0]], shape=(128, 200), dtype=int32) tf.Tensor(
[[9 9 9 ... 0 0 0]
 [2 2 2 ... 0 0 0]
 [2 2 2 ... 0 0 0]
 ...
 [7 7 7 ... 0 0 0]
 [7 7 7 ... 0 0 0]
 [5 5 5 ... 0 0 0]], shape=(128, 200), dtype=int32)
tf.Tensor(
[[39 46 52 ...  0  0  0]
 [50 51  1 ...  0  0  0]
 [34 40 45 ...  0  0  0]
 ...
 [39 65 74 ...  0  0  0]
 [36 79 80 ...  0  0  0]
 [ 3  1 45 ...  0  0  0]], shape=(128, 246), dtype=int32) tf.Tensor(
[[7 7 7 ... 0 0 0]
 [7 7 2 ... 0 0 0]
 [7 7 7 ... 0 0 0]
 ...
 [9 

In [37]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues= np.array([])

# iterate through test set, make predictions based on trained model
for input_example_batch, target_example_batch in ds_series_batch_valid:
    print(input_example_batch)
    pred=model.predict(input_example_batch)
    pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
    y_true=target_example_batch.numpy().flatten()

    preds=np.concatenate([preds,pred_max])
    y_trues=np.concatenate([y_trues,y_true])

# remove padding from evaluation
remove_padding = [(p,y) for p,y in zip(preds,y_trues) if y!=0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p,r_t))
print(classification_report(r_p,r_t))

tf.Tensor(
[[34 49 40 ...  0  0  0]
 [43 46 45 ...  0  0  0]
 [54 65 79 ...  0  0  0]
 ...
 [ 3  1 36 ...  0  0  0]
 [40 66  1 ...  0  0  0]
 [35 81 78 ...  0  0  0]], shape=(128, 228), dtype=int32)


ValueError: in user code:

    /home/rafael/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /home/rafael/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/rafael/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/rafael/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/rafael/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /home/rafael/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:927 __call